В данной задаче нам представлены данные в которых ключевое поле — это то, просрочит ли клиент микро-финансовой организации выплату более чем на 60 дней или нет.
Нам нужно построить модель, которая по входным данным предсказывала бы с максимальной точностью ключевой параметр.
Основной параметр это delinq60plus. Остальные параметры интуитивно понятны из таблиц.

# Цель

Построить модель машинного обучения для предсказания вероятности того, что клиент уйдет в просрочку более 60 дней.

# Задача
осуществить корректную загрузку данных
осуществить разведывательный анализ данных:
- разбить выборку на обучающую и тестовую
- проанализировать пропуски и решить, что с ними делать
- проанализировать выбросы
- создать/ удалить переменные
- закодировать категориальные переменные
- нормализовать числовые переменные (при необходимости)

построить модель и оценить ее качество
при необходимости вернуться к предыдущим шагам, чтобы улучшить качество модели


> Высокая цель. Использовать (не везде, а где мы работаем с данными напрямую) SQL запросы.

In [91]:

import sqlite3

import pandas as pd

pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 320)
pd.set_option('display.width', 80)

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [92]:
def query(q):
    cur = con.cursor()
    cur.execute(q)

In [93]:
df = pd.read_csv('MFOcredit.csv', encoding='cp1251', sep=';')
con = sqlite3.connect('db')

df.to_sql('data', con, index=False, if_exists='replace')


def select(sql):
    return pd.read_sql(sql, con)

35212

### Подготовка данных

In [94]:
select('''SELECT * FROM data''')

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,3,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,4,03-Jan-2013,17-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,5,03-Jan-2013,17-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35207,35208,03-Jan-2014,17-Jan-2014,Мужской,30,Да,Долевая собственность,Гражданский брак/женат/замужем,Да,Официальное,Алтайский край,Да,Да,Нет
35208,35209,03-Jan-2014,17-Jan-2014,Женский,57,Нет,Собственное,Разведен,Да,Пропуск поля,Алтайский край,Да,Нет,Нет
35209,35210,03-Jan-2014,17-Jan-2014,Женский,38,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Новосибирская область,Нет,Да,Нет
35210,35211,03-Jan-2014,17-Jan-2014,Женский,19,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Кемеровская область юг,Нет,Нет,Нет


In [95]:
query("PRAGMA table_info('mfo')")

In [96]:
select('''SELECT date_start FROM data WHERE date_start IS NULL''')

,date_start


Приводим строковые даты к типу DATE и сохраняем в бд как EPOCH time

In [97]:
query(
    '''
    UPDATE data SET
    date_start =
     SUBSTR(date_start, 8) || '-' ||
        CASE SUBSTR (date_start,4, 3)
            WHEN 'Jan' THEN '01'
            WHEN 'Feb' THEN '02'
            WHEN 'Mar' THEN '03'
            WHEN 'Apr' THEN '04'
            WHEN 'May' THEN '05'
            WHEN 'Jun' THEN '06'
            WHEN 'Jul' THEN '07'
            WHEN 'Aug' THEN '08'
            WHEN 'Sep' THEN '09'
            WHEN 'Oct' THEN '10'
            WHEN 'Nov' THEN '11'
            WHEN 'Dec' THEN '12'
        END
     || '-' ||
     SUBSTRING(date_start, 1, 2),
     date_end =
          SUBSTR(date_end, 8) || '-' ||
        CASE SUBSTR (date_end,4, 3)
            WHEN 'Jan' THEN '01'
            WHEN 'Feb' THEN '02'
            WHEN 'Mar' THEN '03'
            WHEN 'Apr' THEN '04'
            WHEN 'May' THEN '05'
            WHEN 'Jun' THEN '06'
            WHEN 'Jul' THEN '07'
            WHEN 'Aug' THEN '08'
            WHEN 'Sep' THEN '09'
            WHEN 'Oct' THEN '10'
            WHEN 'Nov' THEN '11'
            WHEN 'Dec' THEN '12'
        END
     || '-' ||
     SUBSTRING(date_end, 1, 2)
      '''
)
query('''ALTER TABLE data ADD COLUMN date_start_n int;''')
query('''UPDATE data SET date_start_n = UNIXEPOCH(date(date_start));''')
query('''ALTER TABLE data DROP COLUMN date_start''')
query('''ALTER TABLE data RENAME COLUMN date_start_n TO date_start''')

query('''ALTER TABLE data ADD COLUMN date_end_n int;''')
query('''UPDATE data SET date_end_n = UNIXEPOCH(date(date_end));''')
query('''ALTER TABLE data DROP COLUMN date_end''')
query('''ALTER TABLE data RENAME COLUMN date_end_n TO date_end''')


In [98]:
select('''
SELECT * FROM data
''')

,id,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,date_start,date_end
0,1,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет,1357171200,1357948800
1,2,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет,1357171200,1358380800
2,3,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет,1357171200,1358380800
3,4,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет,1357171200,1358380800
4,5,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет,1357171200,1358380800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35207,35208,Мужской,30,Да,Долевая собственность,Гражданский брак/женат/замужем,Да,Официальное,Алтайский край,Да,Да,Нет,1388707200,1389916800
35208,35209,Женский,57,Нет,Собственное,Разведен,Да,Пропуск поля,Алтайский край,Да,Нет,Нет,1388707200,1389916800
35209,35210,Женский,38,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Новосибирская область,Нет,Да,Нет,1388707200,1389916800
35210,35211,Женский,19,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Кемеровская область юг,Нет,Нет,Нет,1388707200,1389916800


Добавляем колонку duration как разницу между date_start и date_end

In [99]:
query('''ALTER TABLE data ADD COLUMN duration_days int''')
query('''UPDATE data SET duration_days = (date_end - date_start)/86400 ''')

Удаляем колонку id

In [100]:
query('''
ALTER TABLE data DROP COLUMN id;
''')

Получаем количество пропусков

In [101]:
select(
    '''
    SELECT
    (select count(*) FROM data WHERE gender = 'Пропуск поля') as gender_gaps,
    (select count(*) FROM data WHERE age = 'Пропуск поля') as age_gaps,
    (select count(*) FROM data WHERE auto = 'Пропуск поля') as auto_gaps,
    (select count(*) FROM data WHERE housing = 'Пропуск поля') as housing_gaps,
    (select count(*) FROM data WHERE marstatus = 'Пропуск поля') as marstatus_gaps,
    (select count(*) FROM data WHERE regclient = 'Пропуск поля') as regclient_gaps,
    (select count(*) FROM data WHERE jobtype = 'Пропуск поля') as jobtype_gaps,
    (select count(*) FROM data WHERE region = 'Пропуск поля') as region_gaps,
    (select count(*) FROM data WHERE credits = 'Пропуск поля') as credits_gaps,
    (select count(*) FROM data WHERE children = 'Пропуск поля') as children_gaps,
    (select count(*) FROM data WHERE date_start = 'Пропуск поля') as date_start_gaps,
    (select count(*) FROM data WHERE date_end = 'Пропуск поля') as date_end_gaps
    FROM data
    GROUP BY 1
    '''
)


,gender_gaps,age_gaps,auto_gaps,housing_gaps,marstatus_gaps,regclient_gaps,jobtype_gaps,region_gaps,credits_gaps,children_gaps,date_start_gaps,date_end_gaps
0,0,0,14077,7603,7564,0,14457,0,946,745,0,0


Пропусков много буду использовать 'Пропуск поля' как дополнительный признак

In [102]:
df = select('SELECT * FROM data')

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35212 entries, 0 to 35211
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         35212 non-null  object
 1   age            35212 non-null  int64 
 2   auto           35212 non-null  object
 3   housing        35212 non-null  object
 4   marstatus      35212 non-null  object
 5   regclient      35212 non-null  object
 6   jobtype        35212 non-null  object
 7   region         35212 non-null  object
 8   credits        35212 non-null  object
 9   children       35212 non-null  object
 10  delinq60plus   35212 non-null  object
 11  date_start     35212 non-null  int64 
 12  date_end       35212 non-null  int64 
 13  duration_days  35212 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 3.8+ MB


In [104]:
df = pd.concat([df,
                pd.get_dummies(df['gender'], prefix='gender'),
                pd.get_dummies(df['auto'], prefix='auto'),
                pd.get_dummies(df['housing'], prefix='housing'),
                pd.get_dummies(df['marstatus'], prefix='marstatus'),
                pd.get_dummies(df['regclient'], prefix='regclient'),
                pd.get_dummies(df['jobtype'], prefix='jobtype'),
                pd.get_dummies(df['region'], prefix='region'),
                pd.get_dummies(df['children'], prefix='children')], axis=1)

df = df.drop(['gender', 'auto', 'housing', 'marstatus', 'regclient',
                         'jobtype', 'region', 'credits', 'children'], axis=1)

df['delinq60plus'] = df['delinq60plus'].map(lambda x: 1 if x == 'Да' else 0)
df

,age,delinq60plus,date_start,date_end,duration_days,gender_Женский,gender_Мужской,auto_Да,auto_Нет,auto_Пропуск поля,housing_Арендуемое,housing_Долевая собственность,housing_Живут с родителями,housing_Муниципальное,housing_Пропуск поля,housing_Собственное,marstatus_Вдова/вдовец,marstatus_Гражданский брак/женат/замужем,marstatus_Пропуск поля,marstatus_Разведен,marstatus_Холост,regclient_Да,regclient_Нет,jobtype_Неофициальное,jobtype_Официальное,jobtype_Пропуск поля,region_Алтайский край,region_Кемеровская область север,region_Кемеровская область юг,region_Красноярский край,region_Новосибирская область,children_Да,children_Нет,children_Пропуск поля
0,44,0,1357171200,1357948800,9,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0
1,21,0,1357171200,1358380800,14,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0
2,25,0,1357171200,1358380800,14,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0
3,47,0,1357171200,1358380800,14,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
4,22,0,1357171200,1358380800,14,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35207,30,0,1388707200,1389916800,14,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0
35208,57,0,1388707200,1389916800,14,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0
35209,38,0,1388707200,1389916800,14,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0
35210,19,0,1388707200,1389916800,14,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0


In [105]:
# Разделение данных на обучающую и тестовую выборки
y = df.delinq60plus
X = df.drop(['delinq60plus'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

# Модель RandomForest
forest = RandomForestClassifier(n_estimators=800, max_depth=17,
                                random_state=2020, n_jobs=-1)
my_pipeline = Pipeline(steps=[('model', forest)])
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_test)

Pipeline(steps=[('model',
                 RandomForestClassifier(max_depth=17, n_estimators=800,
                                        n_jobs=-1, random_state=2020))])

In [106]:
# Оценка модели
score_auc = roc_auc_score(y_test, preds)
print('AUC: ', score_auc)
forest.fit(X_train, y_train)

AUC:  0.7514358110704


RandomForestClassifier(max_depth=17, n_estimators=800, n_jobs=-1,
                       random_state=2020)

In [107]:
print('AUC на обучающей выборке: {:.3f}'.format(roc_auc_score(y_train, forest.predict_proba(X_train)[:, 1])))
print('AUC на контрольной выборке: {:.3f}'.format(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 0.989
AUC на контрольной выборке: 0.843


### Наивный Байес

In [108]:
from sklearn.naive_bayes import GaussianNB

gaus_nb = GaussianNB()

gaus_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.format(roc_auc_score(y_train, gaus_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.format(roc_auc_score(y_test, gaus_nb.predict(X_test))))

GaussianNB()

AUC на обучающей выборке: 0.721
AUC на контрольной выборке: 0.718


### Мульти номинальный Наивный Бейс

In [109]:
from sklearn.naive_bayes import MultinomialNB

multinominal_nb = MultinomialNB()

multinominal_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.format(roc_auc_score(y_train, multinominal_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.format(roc_auc_score(y_test, multinominal_nb.predict(X_test))))

MultinomialNB()

AUC на обучающей выборке: 0.527
AUC на контрольной выборке: 0.531


In [110]:
### Complement Naive Bayes

In [111]:
from sklearn.naive_bayes import ComplementNB

complement_nb = ComplementNB()

complement_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.format(roc_auc_score(y_train, complement_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.format(roc_auc_score(y_test, complement_nb.predict(X_test))))

ComplementNB()

AUC на обучающей выборке: 0.534
AUC на контрольной выборке: 0.539


In [112]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression(fit_intercept=True)

linear_regression.fit(X_train, y_train)

print(f'AUC на обучающей выборке: {roc_auc_score(y_train, linear_regression.predict(X_train))}')
print(f'AUC на контрольной выборке: {roc_auc_score(y_test, linear_regression.predict(X_test))}')

LinearRegression()

AUC на обучающей выборке: 0.832245147695714
AUC на контрольной выборке: 0.8281484623951112
